<a href="https://colab.research.google.com/github/SnehaDharne/GDM-diagnosis/blob/main/Classifiers_with_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
path='/content/drive/MyDrive/datasets for colab/pima-indians-diabetes.csv'

In [ ]:
pip install vecstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19879 sha256=1b18596fe868c9a491694275128bfea03cd4fc8f6806a581591d4de05f7a92a9
  Stored in directory: /root/.cache/pip/wheels/17/89/0b/21d5484cbf713c95b641ec1bdc40dd7ae798cbdea2337e3535
Successfully built vecstack


This notebook includes Classifiers with wrapper method for:

1. LR
2. SVM
3. RF

In [ ]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats



from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings('ignore')

from vecstack import stacking

from sklearn.feature_selection import SequentialFeatureSelector as SFS

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score

In [ ]:
df=pd.read_csv(path)
# make age category column which will help age-appropriate in filling values which are 0
df['age_category'] = pd.cut(df['Age'], bins=[0, 21, 31, 41, 51, 61, 71, 81, np.inf], labels=[1,2,3,4,5,6,7,8])
df['age_category'] = df['age_category'].astype(int)
agecat = [1,2,3,4,5,6,7,8]


features = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI','Insulin']
for feature in features:
  for agec in agecat:
        if df[feature].dtype == 'float64':
            df[feature].replace(0, round(df[df['age_category'] == agec][feature].mean(), 1), inplace=True)
        else:
            df[feature].replace(0, round(df[df['age_category'] == agec][feature].mean(), 0), inplace=True)





z= np.abs(stats.zscore(df))

threshold = 3

df = df[(z < 3).all(axis=1)]

q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr= q3-q1


df = df[~((df <(q1 - 1.5 * iqr))|(df>(q3+1.5*iqr))).any(axis=1)]

X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)




In [ ]:
#@title LR Hyperparameter
# parameter grid
parameters = {
    'penalty' : ['l1','l2','elastinet'],    #‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear','sag','saga'],
}

logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=10)

clf.fit(X_train,y_train)
#lr=(solver='liblinear', C=10.0, penalty='l1')
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)

Tuned Hyperparameters : {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.8058080808080808


In [ ]:
#@title Best parameters LR
print(clf.best_params_)
clf_predictions = clf.predict(X_test)
print(classification_report(y_test, clf_predictions))
mean_squared_error(y_test,clf_predictions)

{'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
              precision    recall  f1-score   support

           0       0.80      0.97      0.88        79
           1       0.87      0.41      0.55        32

    accuracy                           0.81       111
   macro avg       0.83      0.69      0.72       111
weighted avg       0.82      0.81      0.79       111



0.1891891891891892

In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
lr=LogisticRegression(solver='liblinear', C=10.0, penalty='l1')
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88        79
           1       0.87      0.41      0.55        32

    accuracy                           0.81       111
   macro avg       0.83      0.69      0.72       111
weighted avg       0.82      0.81      0.79       111



In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
sfs_lr_train= SFS(lr,n_features_to_select= 4,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_lr_train=sfs_lr_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_lr_train.get_feature_names_out(input_features=None)).T



,0,1,2,3
0,Glucose,BMI,DiabetesPedigreeFunction,age_category


In [ ]:
sfs_lr_train= SFS(lr,n_features_to_select= 3,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_lr_train=sfs_lr_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_lr_train.get_feature_names_out(input_features=None)).T

,0,1,2
0,Glucose,BMI,age_category


In [ ]:


df2 = df.filter(['BMI','Glucose','DiabetesPedigreeFunction','age_category','Outcome'], axis=1)
df3 = df.filter(['BMI','Glucose','age_category','Outcome'], axis=1)


X = df2.drop(['Outcome'], axis=1)
y = df2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
lr_feat_sel=LogisticRegression(solver='liblinear', C=10.0, penalty='l1')
lr_feat_sel.fit(X_train,y_train)
y_pred=lr_feat_sel.predict(X_test)
print('All 4',classification_report(y_test, y_pred))

X = df3.drop(['Outcome'], axis=1)
y = df3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
lr_feat_sel=LogisticRegression(solver='liblinear', C=10.0, penalty='l1')
lr_feat_sel.fit(X_train,y_train)
y_pred=lr_feat_sel.predict(X_test)
print('Top 3 ',classification_report(y_test, y_pred))




All 4               precision    recall  f1-score   support

           0       0.80      0.94      0.87        79
           1       0.74      0.44      0.55        32

    accuracy                           0.79       111
   macro avg       0.77      0.69      0.71       111
weighted avg       0.78      0.79      0.77       111

Top 3                precision    recall  f1-score   support

           0       0.83      0.96      0.89        79
           1       0.84      0.50      0.63        32

    accuracy                           0.83       111
   macro avg       0.83      0.73      0.76       111
weighted avg       0.83      0.83      0.81       111



In [ ]:
#@title HyperParamters SVM
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC
# fitting the model for grid search

clf_svm=svm.SVC()

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(clf_svm, param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.708 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.708 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
#@title Best parameters SVM
print(grid.best_params_)
grid_predictions = grid.predict(X_test)
print(classification_report(y_test, grid_predictions))
mean_squared_error(y_test,grid_predictions)

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.81      0.97      0.89        79
           1       0.88      0.44      0.58        32

    accuracy                           0.82       111
   macro avg       0.84      0.71      0.73       111
weighted avg       0.83      0.82      0.80       111



0.18018018018018017

In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
svc=svm.SVC(gamma=0.0001, C=10, kernel='rbf')
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.97      0.89        79
           1       0.88      0.44      0.58        32

    accuracy                           0.82       111
   macro avg       0.84      0.71      0.73       111
weighted avg       0.83      0.82      0.80       111



In [ ]:
sfs_svm_train= SFS(svc,n_features_to_select= 4,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_svm_train=sfs_svm_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_svm_train.get_feature_names_out(input_features=None)).T

,0,1,2,3
0,Pregnancies,Glucose,BMI,Age


In [ ]:
sfs_svm_train= SFS(svc,n_features_to_select= 4,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_svm_train=sfs_svm_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_svm_train.get_feature_names_out(input_features=None)).T

,0,1,2,3
0,Pregnancies,Glucose,BMI,Age


In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
sfs_svm_train= SFS(svc,n_features_to_select= 3,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_svm_train=sfs_svm_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_svm_train.get_feature_names_out(input_features=None)).T

,0,1,2
0,Pregnancies,Glucose,Age


In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
df2 = df.filter(['Age','Glucose','Pregnancies','Outcome'], axis=1)
df3 = df.filter(['BMI','Glucose','Pregnancies','Outcome','Age'], axis=1)


X = df3.drop(['Outcome'], axis=1)
y = df3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
svm_feat_sel=svm.SVC(gamma=0.0001, C=10, kernel='rbf')
svm_feat_sel.fit(X_train,y_train)
y_pred=svm_feat_sel.predict(X_test)
print('Top 4:',classification_report(y_test, y_pred))

X = df2.drop(['Outcome'], axis=1)
y = df2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
svm_feat_sel=svm.SVC(gamma=0.0001, C=10, kernel='rbf')
svm_feat_sel.fit(X_train,y_train)
y_pred=svm_feat_sel.predict(X_test)
print('Top 3:',classification_report(y_test, y_pred))

Top 4:               precision    recall  f1-score   support

           0       0.78      0.96      0.86        79
           1       0.79      0.34      0.48        32

    accuracy                           0.78       111
   macro avg       0.78      0.65      0.67       111
weighted avg       0.78      0.78      0.75       111

Top 3:               precision    recall  f1-score   support

           0       0.79      0.99      0.88        79
           1       0.92      0.34      0.50        32

    accuracy                           0.80       111
   macro avg       0.85      0.67      0.69       111
weighted avg       0.83      0.80      0.77       111



In [ ]:
#@title RF Hyperparameter
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
rf = RandomForestClassifier(n_jobs=-1 ,n_estimators=100, oob_score = True)
param_grid = {
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
rfc.fit(X_train,y_train)
print(classification_report(y_test,rfc.predict(X_test)))

#rf = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=100, oob_score = True, max_features)
print(rfc.best_params_)
rfc_predictions = rfc.predict(X_test)
mean_squared_error(y_test,rfc_predictions)


              precision    recall  f1-score   support

           0       0.85      0.91      0.88        79
           1       0.73      0.59      0.66        32

    accuracy                           0.82       111
   macro avg       0.79      0.75      0.77       111
weighted avg       0.81      0.82      0.81       111

{'max_features': 'sqrt', 'n_estimators': 700}


0.18018018018018017

In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
rf = RandomForestClassifier(n_jobs=-1 ,n_estimators=700, oob_score = True, max_features='sqrt')
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88        79
           1       0.75      0.56      0.64        32

    accuracy                           0.82       111
   macro avg       0.79      0.74      0.76       111
weighted avg       0.81      0.82      0.81       111



In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
sfs_rf_train= SFS(rf,n_features_to_select= 4,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_rf_train=sfs_rf_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_rf_train.get_feature_names_out(input_features=None)).T

,0,1,2,3
0,Pregnancies,Glucose,Insulin,age_category


In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
sfs_rf_train= SFS(rf,n_features_to_select= 4,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_rf_train=sfs_rf_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_rf_train.get_feature_names_out(input_features=None)).T

,0,1,2,3
0,Pregnancies,Glucose,BMI,Age


In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
sfs_rf_train= SFS(rf,n_features_to_select= 3,direction='forward',scoring='accuracy',cv=5,n_jobs=-1)
sfs_rf_train=sfs_rf_train.fit(X_train,y_train)
pd.DataFrame.from_dict(sfs_rf_train.get_feature_names_out(input_features=None)).T

,0,1,2
0,Glucose,BMI,Age


In [ ]:

df2 = df.filter(['BMI','Glucose','Pregnancies','Age','Outcome'], axis=1)
X = df2.drop(['Outcome'], axis=1)
y = df2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
rf_feat_sel=RandomForestClassifier(n_jobs=-1 ,n_estimators=700, oob_score = True, max_features='sqrt')
rf_feat_sel.fit(X_train,y_train)
y_pred=rf_feat_sel.predict(X_test)
print('top 4:',classification_report(y_test, y_pred))

df3 = df.filter(['BMI','Glucose','Age','Outcome'], axis=1)
X = df3.drop(['Outcome'], axis=1)
y = df3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
rf_feat_sel=RandomForestClassifier(n_jobs=-1 ,n_estimators=700, oob_score = True, max_features='sqrt')
rf_feat_sel.fit(X_train,y_train)
y_pred=rf_feat_sel.predict(X_test)
print('top 3:',classification_report(y_test, y_pred))

top 4:               precision    recall  f1-score   support

           0       0.83      0.91      0.87        79
           1       0.71      0.53      0.61        32

    accuracy                           0.80       111
   macro avg       0.77      0.72      0.74       111
weighted avg       0.79      0.80      0.79       111

top 3:               precision    recall  f1-score   support

           0       0.83      0.91      0.87        79
           1       0.71      0.53      0.61        32

    accuracy                           0.80       111
   macro avg       0.77      0.72      0.74       111
weighted avg       0.79      0.80      0.79       111

